In [1]:
import pandas as pd #analyzing, cleaning, and storing data 
import io
import os 
import pyarrow.parquet as pq #parquet specific analysis 
import numpy as np
import stats
import fastparquet #make sure that this is installed in your anaconda environment
from scipy import stats
from scipy.stats import median_abs_deviation
from scipy.stats import zscore
import warnings 
warnings.filterwarnings("ignore")
from sklearn.datasets import make_classification
from feature_engine.selection import DropCorrelatedFeatures

In [2]:
#call the folder with multiple parquet files

In [3]:
from pathlib import Path
# the r below must be placed before the file to indicate that it is a raw string - // are not escape sequences but rather characters 
data_directory = Path(r'C:\Users\abbyk\OneDrive - Northeastern University\Documents\Capstone\Multiple Parquets for Practice\B040103a.parquet')

In [4]:
# concatenate multiple parquet files into one cvs file

In [5]:
# full_df = pd.concat(
#    pd.read_parquet(parquet_file)
#    for parquet_file in data_directory.glob('*.parquet') #.glob searches for only parquets
# )

In [6]:
full_df = pd.read_parquet(data_directory, engine='fastparquet')

In [7]:
# create the CSV file

In [8]:
# full_df.to_csv('original_data.csv')

In [9]:
#practice: remove one column and update csv file, the fifth column 'cell area shape area'
  
# read the CSV file
# original_data = pd.read_csv('original_data.csv')

# drop function which is used in removing or deleting rows or columns from the CSV files
# original_data.drop('Cells_AreaShape_Area', inplace=True, axis=1)

# print into a new csv file
# original_data.to_csv('new_data.csv')

In [10]:
# normalize the data using the equation for robust z-transform 

# axis = 0 calculates the median for each column seperately

numeric_df = full_df._get_numeric_data() #get numerical values only
full_df_normalized = numeric_df

In [11]:
# this is how you would do it by hand, but it doesn't work

# median = np.median(numeric_df) #take the median
# absolute_deviation = np.abs(numeric_df - median) #take the absolute value
# MAD = np.median(absolute_deviation) #calculate median absolute deviation 
# full_df_normalized = (0.6745 * (numeric_df - numeric_df.median(axis=0))) / MAD

In [12]:
# remove the outliers 

n_MAD = 500
threshold = 0.6745 * n_MAD

# remove any columns with NaN values
full_df_normalized = (0.6745 * (full_df_normalized- full_df_normalized.median(axis=0))) / stats.median_abs_deviation(full_df_normalized, axis=0)
full_df_normalized = full_df_normalized.dropna(axis=1)
# drop columns with zeros
# full_df_normalized = full_df_normalized.loc[(full_df_normalized!=0).any(axis=1)]
full_df_normalized.shape

(384, 4659)

In [13]:
rows_to_keep = (np.abs(full_df_normalized) < threshold).all(axis=1)
full_df_normalized = full_df_normalized[rows_to_keep] #YOU ARE THE PROBLEM
full_df_normalized.shape

(375, 4659)

In [14]:
# remove problematic columns as per ryan's work 
# Create a boolean (T/F) array indicating which columns contain the key word
cols_to_drop = full_df_normalized.columns[full_df_normalized.columns.str.contains('BoundingBox|Center|EulerNumber|Count|Neighbors|FormFactor|Image|Location|Overflow|mito_tubeness|Manders|MinIntensity|Cytoplasm_Texture_Contrast_DNA_10|Cytoplasm_Texture_DifferenceEntropy_DNA_10|Cytoplasm_Texture_Entropy_DNA_10|Cytoplasm_Texture_InfoMeas1_DNA_10|Cytoplasm_Texture_InfoMeas2_DNA_10|Cytoplasm_Texture_SumEntropy_DNA_10|Cytoplasm_Texture_SumVariance_DNA_10|Cytoplasm_Texture_Variance_DNA_10|CellsIncludingEdges')]

full_df_normalized.drop(cols_to_drop, axis=1, inplace=True)
# cuts slightly less columns than ryan's work
full_df_normalized.shape
full_df_normalized.to_csv('full_df_normalized.csv')

In [15]:
# compute the correlation matrix of numeric-only quantities with pearson correlation
corr_matrix = full_df_normalized.corr().abs()
# convert correlation values to a csv file
# correlation_matrix.to_csv('correlation_values.csv')

In [211]:
corr = DropCorrelatedFeatures(variables=None, method = 'spearman', threshold = 0.9)
dropped_correlation = corr.fit_transform(full_df_normalized)
# obtain the correlated feature groups that are stored in the transformer’s attributes:
groups = corr.correlated_feature_sets_

In [50]:
dropped_correlation.to_csv('dropped_correlation.csv')

In [236]:
groups = sorted(groups)

In [248]:
# Define the query 
query = 'Cytoplasm_AreaShape_Zernike_3_3'

# Initialize a flag to check if the string has been found
found = False
position = 0

# sort the appropriate set 
for i, item_set in enumerate(groups):
    if query in item_set:
        found = True
        position = i
        set_list = sorted(groups[position])
        location = set_list.index(query)
        print(f"The query you entered is position '{location+1}' in set {position+1}")

The query you entered is position '2' in set 4
